In [92]:
from openai import OpenAI
import os
import asyncio
from pyppeteer import launch
import nest_asyncio
from pypdf import PdfReader
from datetime import datetime

In [93]:
# read current resume
def read_current_resume():
    reader = PdfReader("CV Yusuf Surya - 8-5-2024.pdf")
    number_of_pages = len(reader.pages)
    texts = ""
    for page_number in range(number_of_pages):
        page = reader.pages[page_number]
        text = page.extract_text()
        texts = texts + f"Page {page_number + 1}:\n{text}\n---"
    return texts

In [94]:
# read job description txt file for now
def read_job_description():
    with open("job_data.txt", "r") as file:
        return file.read()

In [95]:
def model_selection(model):
    if model == "openai":
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        ai_model = "gpt-4o"
    elif model == "deepseek":
        client = OpenAI(api_key=os.getenv("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")
        ai_model = "deepseek-chat"
    else:
        raise Exception("Invalid model")
    return client, ai_model

In [96]:
# read cover letter draft
def read_cover_letter_draft():
    reader = PdfReader("About me.pdf")
    number_of_pages = len(reader.pages)
    texts = ""
    for page_number in range(number_of_pages):
        page = reader.pages[page_number]
        text = page.extract_text()
        texts = texts + f"Page {page_number + 1}:\n{text}\n---"
    return texts

In [97]:
# Work Experience
def cover_letter_txt(client, ai_model, jobdesc, resume, about_me, address):
    today_date = datetime.today().strftime('%B %d, %Y')
    response = client.chat.completions.create(
        model=ai_model,
        messages=[
            {"role": "system", "content": "You are human expert in cover letter writing. Use simple language and write like you mean it."},
            {"role": "user", "content": f"Job decsription: {jobdesc}"},
            {"role": "user", "content": f"Resume: {resume}"},
            {"role": "user", "content": f"About me: {about_me}"},
            {"role": "user", "content": f"Today's date: {today_date}. My address: {address}"},
            {"role": "user", "content": "Use my personal information"},
            {"role": "user", "content": "Make the opening more engaging, arrange story with high specificity in skills, stronger call to action. Only response with the cover letter no need explanation."},
            ],
        stream=False
    )

    cl = response.choices[0].message.content
    return cl

In [98]:
# Cover letter to html
def cover_letter_html(client, ai_model, cover_letter, html_format):
    
    response = client.chat.completions.create(
        model=ai_model,
        messages=[
            {"role": "system", "content": "You are a expert in html language."},
            {"role": "user", "content": "Format the cover letter to html format."},
            {"role": "user", "content": f"Cover Letter: {cover_letter}"},
            {"role": "user", "content": f"Guidelines: {html_format}"},
            {"role": "user", "content": "Make sure the html is formatted correctly and 1 letter page long."},
            {"role": "user", "content": "Font size should be max 12px and font family should be Arial."},
            {"role": "user", "content": "Do not include your explanation in the output."},
            ],
        stream=False
    )

    html_file = response.choices[0].message.content
    clean_html = html_file.replace("```html", "")
    clean_html = clean_html.replace("```", "")
    return clean_html

In [99]:
def html_to_pdf(clean_html, name):

    nest_asyncio.apply()

    async def generate_pdf_from_html(html_content, pdf_path):
        browser = await launch()
        page = await browser.newPage()
        
        await page.setContent(html_content)
        
        await page.pdf({
            'path': pdf_path,
            'format': 'A4',
            'margin': {
                'top': '0.5in',
                'right': '0.5in',
                'bottom': '0.5in',
                'left': '0.5in'
            },
            'printBackground': True
        })
        
        await browser.close()

    # HTML content
    html_content = clean_html

    # Run the function
    asyncio.get_event_loop().run_until_complete(generate_pdf_from_html(html_content, f'{name}_Cover_Letter.pdf'))

In [100]:
address = "208 N Homewood Ave, Pittsburgh, PA 15208"
html_format = """
    Guidelines:
    Use Semantic HTML

    Wrap the cover letter in <section> and <div> tags.
    Use <h3> for the candidate’s name, <h3> for section titles, and <p> or <ul> for content.

    Ensure PDF-Friendly Styling

    Set body { margin: 0; padding: 20px; } to prevent extra margins when rendering PDFs.
    Use page-break-before: always; where necessary to manage page flow.
    Set max-width: 800px; to keep content properly aligned.
    """

In [101]:
client, ai_model = model_selection("openai")
jobdesc = read_job_description()
resume = read_current_resume()
about_me = read_cover_letter_draft()
cover_letter = cover_letter_txt(client, ai_model, jobdesc, resume, about_me, address)
print(cover_letter)

Ignoring wrong pointing object 6 0 (offset 0)


Yusuf Surya  
208 N Homewood Ave  
Pittsburgh, PA 15208  
+1 412-579-2443  
ysurya@andrew.cmu.edu  
March 11, 2025  

Hiring Manager  
SESCO  

Dear Hiring Manager,

I'm writing to express my enthusiasm for the Quantitative Trading Analyst position at SESCO. I've always been captivated by the dynamic nature of electricity markets, and your forward-thinking approach resonates with my professional journey and aspirations.

Growing up in Indonesia, an archipelago of untold possibilities, I developed a keen interest in problem-solving and adaptability. These traits guided my transition from Petroleum Engineering to becoming a business analyst at Markplus Inc. Here, I honed my skills in market research, achieving an 80% success rate in client proposal deals within a year, demonstrating my ability to quickly grasp and apply complex business models.

Currently, as a Data Analyst at the Ministry of Finance in Indonesia, I've led projects of vast scale, such as reassessing over a million govern

In [102]:
client, ai_model = model_selection("deepseek")
html_cover_letter = cover_letter_html(client, ai_model, cover_letter, html_format)
print(html_cover_letter)


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Cover Letter - Yusuf Surya</title>
    <style>
        body {
            margin: 0;
            padding: 20px;
            font-family: Arial, sans-serif;
            font-size: 12px;
            max-width: 800px;
        }
        h3 {
            font-size: 14px;
            margin-bottom: 10px;
        }
        p, ul {
            margin: 0 0 10px 0;
            line-height: 1.5;
        }
        .section-title {
            font-weight: bold;
            margin-top: 20px;
        }
        .contact-info {
            margin-bottom: 20px;
        }
        .signature {
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <section>
        <div class="contact-info">
            <h3>Yusuf Surya</h3>
            <p>208 N Homewood Ave</p>
            <p>Pittsburgh, PA 15208</p>
            <p>+1 412-579-2443</p>
 

In [103]:
html_to_pdf(html_cover_letter, "SESCO - Yusuf_Surya")